# Visualizing Convolutional Layers in a Trained VGG Network

### What have the various feature maps in a CNN been trianed to look for?

## Description

---

Convolutional Neural Nets (CNNs) are very deep data sructutes that extract increasingly complex spacial patterns in 2d or 3d data.

A CNN trained on a large corpus of images will build representations of the training at different levels of complexity at different layers of the network. In theory, lower layers will build representations of low level features such as colors and edges. Then higer levels will combine thos representations into increasingly complex feature detectors, e.g. edges -> corners -> squares -> doors -> houses

This ability for CNNs to find patterns within patterns within patterns makes them extrmely powerful, but also extremly mysterious. Is there a way to increase the interpretability of what CNNs are doing as an image is fed forward through the network?

## Methods

---

In this exploration, I will be using a pretrianed VGG16 network wo build an image that most intensly 'excites' a subset of feature maps in specific convolutional layers.

The convolutional layers I will be examining are:
- Conv1_1 with 64 feature maps to explore
- Conv2_1 with 128 feature maps to explore
- Conv3_1 with 256 feature maps to explore
- Conv4_1 with 512 feature maps to explore
- Conv5_1 with 512 feature maps to explore

<img src="./assets/readme_a.png" width="300"/>

I do this by starting with a targer tensor of random noise and feeding it through the pretrained VGG network which has had it's parameters frozen. At a specific feature map in the layer I am examining, I grab the activations calculate a loss by comparing the actual activation tensor to a target activation that is unreachably high (In this case a target tensor filled with the maximum squared value in the actual tensor). I then backpropogate the loss to update the target tensor; In this way after each iteration it continually updates into an input that maximally actucated the feature map in question.

I then display the tensor as an image for an idea of what that particualr feature map has been trianed to look for! 

### Imports

In [1]:
import torch
import torch.optim as optim
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### Defining the Model

In [2]:

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

vgg = torchvision.models.vgg16(pretrained=True).to(device)

for param in vgg.parameters():
    param.requires_grad = False

vgg.activation = {}
def get_activation(name):
    def hook(model, input, output):
        vgg.activation[name] = output.squeeze()
    return hook
# mapping between layers indexed in model and layer names
layers = {'conv1_1': 0,
          'conv2_1': 5, 
          'conv3_1': 10, 
          'conv4_1': 17,
          'conv5_1': 24,}

vgg.features[layers['conv1_1']].register_forward_hook(get_activation('Conv1_1'))
vgg.features[layers['conv2_1']].register_forward_hook(get_activation('Conv2_1'))
vgg.features[layers['conv3_1']].register_forward_hook(get_activation('Conv3_1'))
vgg.features[layers['conv4_1']].register_forward_hook(get_activation('Conv4_1'))
vgg.features[layers['conv5_1']].register_forward_hook(get_activation('Conv5_1'))

### Helpers

In [3]:
def load_image(img_path):
    image = Image.open(img_path).convert('RGB')
    in_transform = transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.485, 0.456, 0.406), 
                                             (0.229, 0.224, 0.225))])
    image = in_transform(image).unsqueeze(0)
    
    return image

In [4]:
def im_convert(tensor):
    """ Display a tensor as an image. """
    
    image = tensor.to("cpu").clone().detach()
    image = image.numpy().squeeze()
    image = image.transpose(1,2,0)
    image = image * np.array((0.229, 0.224, 0.225)) + np.array((0.485, 0.456, 0.406))
    image = image.clip(0, 1)

    return image

### Putting it all together

In [5]:
def show(layer_to_visualize='Conv1_1',
        filers_to_show=(2,2),
        resolution=100,
        steps=200, lr=0.01,
        shift=0, dist=(-1., 1.),
        random_state=None):

  random = np.random.RandomState(random_state) if random_state is not None else np.random
  fig, axs = plt.subplots(filers_to_show[0],
                          filers_to_show[1],
                          figsize=(filers_to_show[1]*2,filers_to_show[0]*2+0.3),
                          constrained_layout=True)
  if hasattr(axs, '__len__') == False:
    axs = np.array([axs])
  axs = axs.reshape(filers_to_show[0], filers_to_show[1])

  for x in range(filers_to_show[0]):
    for y in range(filers_to_show[1]):
      map_number = (x * (filers_to_show[1])) + y + shift
      target_image = torch.from_numpy(random.uniform(*dist,
                                      size=(3,resolution,resolution))).unsqueeze(0)
      target = target_image.clone().type(torch.FloatTensor).to(device).requires_grad_(True)
      optimizer = optim.Adam([target], lr=lr)

      vgg.eval()

      for ii in range(1, steps+1):
        vgg.forward(target)
        output = vgg.activation[layer_to_visualize][map_number]

        out = output.detach()
        expected = np.empty((*list(out.size()),))
        expected.fill(float(torch.max(out)))
        expected = expected**2
        expected = torch.from_numpy(expected).to(device).requires_grad_(False)

        loss = torch.mean((output - expected)**2)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

      axs[x][y].imshow(im_convert(target))
      axs[x][y].axis('off')
      axs[x][y].annotate(map_number,
            xy=(0, 0), color='white',
            fontweight='bold',
            verticalalignment='top',
            bbox=dict(boxstyle="round", fc="black"))
  
  fig.suptitle(layer_to_visualize, color='white', fontweight='bold')
  fig.patch.set_facecolor('black')
  plt.show()


### Testing it out

In [ ]:
show('Conv1_1', (2,5), resolution=100, shift=20, random_state=100, dist=(0., 2.))
show('Conv2_1', (2,5), resolution=110, shift=20, random_state=100, dist=(0., 0.5))
show('Conv3_1', (2,5), resolution=120, shift=20, random_state=100, dist=(-0.1, 0.2))
show('Conv4_1', (2,5), resolution=140, shift=20, random_state=100, dist=(-0.2, 0.2))
show('Conv5_1', (2,5), resolution=160, shift=25, random_state=100, dist=(-0.25, 0.25))

## Results

---

Overall the results are quite impressive. You can obseve that in lower levels such as Conv1_1 feature maps are activated by relitivly solid colors and simple lines and shapes. I can deduce that this layer's feature maps have, over the course of training on the images dataset, learned to become simple color and edge detectors.

This is in contrast to the highest layers which are activated by complicated and abstract patterns. These generated images are neat to look at, but some have enough form to actually guess what high level feature they are detecting!
For exapmle, I found the 25th feature map of the Conv5_1 layer to be excited specifically by patterns that resemble human eyes! do you see any eyes staring back at you grom this generated image?

<img src="./assets/output1.png" width="200"/>

I can also see what resemble animal eyes in the 116th feature map of the Conv5_1 layer and what strike me as perhaps birds/parakeets in the 104th feature map of the Conv5_1 layer

<img src="./assets/output2.jpeg" width="400"/>

You can also observe blank and noisy squares generated for some feature maps, which appear to be more frequent in higher layers.
At first I thought that it might mean that feature map did not actually learn anything useful during training, but i think the reality is that this method of backpropogation used to sensitive to starting conditions. I suspect that higher layers have a harder time finding the complex pattern that excites thim in the noisy starting tensor and never converge to anything meaningful. 
